# Comparación de Modelos con Validación Cruzada
## Pipeline Completo: XGBoost, Random Forest y Neural Network

**Objetivo**: Comparar el desempeño de múltiples modelos de clasificación para detección de lavado de activos usando:
- **Validación cruzada estratificada** (k=5)
- **SMOTE** aplicado dentro de cada fold (sin data leakage)
- **Métricas enfocadas en fraude**: Recall, F1-score, AUC-ROC

Modelos evaluados:
1. **XGBoost**: Gradient Boosting (ensemble)
2. **Random Forest**: Bagging ensemble
3. **Neural Network**: Feedforward MLP (PyTorch)

In [ ]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

# Módulos del proyecto
from src.utils.config import load_config
from src.utils.reproducibility import set_seed
from src.models.cross_validation import stratified_cv_with_smote, compare_models_cv
from src.models.evaluation import compare_models, plot_model_comparison

# Configuración
config = load_config('../../configs/config.yaml')
set_seed(config['project']['random_seed'])

print("✓ Librerías cargadas")
print(f"  Random seed: {config['project']['random_seed']}")

## 1. Carga y Preprocesamiento

In [ ]:
# Carga datos
df = pd.read_csv('../../data/synthetic/aml_colombia_synthetic.csv')
print(f"📊 Dataset: {df.shape}")
print(f"   Fraude: {df['isFraud'].sum()} ({df['isFraud'].mean()*100:.2f}%)")

# Encode tipo
le = LabelEncoder()
df['type_encoded'] = le.fit_transform(df['type'])

# Features
features = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
           'oldbalanceDest', 'newbalanceDest', 'type_encoded']

X = df[features].values
y = df['isFraud'].values

# Scale (importante para NN)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\n✓ Features: {X.shape}")
print(f"  Columnas: {features}")

## 2. Definición de Modelos

In [ ]:
# Parámetros de configuración
xgb_params = config['xgboost']
rf_params = config['random_forest']
cv_params = config['cross_validation']
smote_params = config['smote']

# Calcula weight para modelos que lo necesitan
scale_pos_weight = (y == 0).sum() / (y == 1).sum()

# Funciones que retornan modelos instanciados
def create_xgboost():
    return xgb.XGBClassifier(
        n_estimators=xgb_params['n_estimators'],
        max_depth=xgb_params['max_depth'],
        learning_rate=xgb_params['learning_rate'],
        subsample=xgb_params['subsample'],
        colsample_bytree=xgb_params['colsample_bytree'],
        scale_pos_weight=scale_pos_weight if xgb_params['use_scale_pos_weight'] else 1,
        random_state=config['model']['random_state'],
        tree_method='hist',
        device='cpu'  # CPU para compatibilidad
    )

def create_random_forest():
    return RandomForestClassifier(
        n_estimators=rf_params['n_estimators'],
        max_depth=rf_params['max_depth'],
        min_samples_split=rf_params['min_samples_split'],
        min_samples_leaf=rf_params['min_samples_leaf'],
        class_weight=rf_params['class_weight'],
        random_state=config['model']['random_state'],
        n_jobs=-1
    )

print("✓ Modelos definidos:")
print("  1. XGBoost")
print("  2. Random Forest")
print(f"\n✓ Validación cruzada: {cv_params['n_folds']}-fold estratificada")
print(f"✓ SMOTE: sampling_strategy={smote_params['sampling_strategy']}")

## 3. Validación Cruzada con SMOTE

**IMPORTANTE**: SMOTE se aplica SOLO dentro de cada fold de entrenamiento, nunca en validación, para evitar data leakage.

In [ ]:
# Diccionario de modelos
models = {
    'XGBoost': create_xgboost,
    'Random Forest': create_random_forest
}

# Ejecuta comparación con CV
cv_results = compare_models_cv(
    X_scaled, y,
    models=models,
    n_folds=cv_params['n_folds'],
    smote_params=smote_params,
    random_state=config['model']['random_state']
)

## 4. Resultados de Validación Cruzada

In [ ]:
# Muestra tabla comparativa
print("\n" + "="*80)
print("RESULTADOS DE VALIDACIÓN CRUZADA (5-Fold con SMOTE)")
print("="*80)
print(cv_results)
print("="*80)

# Identifica mejor modelo por F1-score (métrica clave para fraude)
best_model = cv_results['f1_score'].idxmax()
best_f1 = cv_results.loc[best_model, 'f1_score']

print(f"\n🏆 MEJOR MODELO: {best_model}")
print(f"   F1-Score: {best_f1:.4f}")
print(f"   Recall: {cv_results.loc[best_model, 'recall']:.4f}")
print(f"   AUC-ROC: {cv_results.loc[best_model, 'auc_roc']:.4f}")

## 5. Visualización de Resultados

In [ ]:
# Convierte resultados a formato para visualización
results_dict = cv_results.to_dict('index')

# Gráfico comparativo
fig, ax = plot_model_comparison(
    results_dict,
    metrics=['accuracy', 'precision', 'recall', 'f1_score', 'auc_roc'],
    figsize=(14, 6)
)
plt.savefig('../../reports/figures/model_comparison_cv.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Gráfico de barras individual por métrica
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'auc_roc']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6']

for i, metric in enumerate(metrics):
    cv_results[metric].plot(kind='bar', ax=axes[i], color=colors[i], alpha=0.7)
    axes[i].set_title(f'{metric.replace("_", " ").title()}', 
                     fontsize=12, weight='bold')
    axes[i].set_xlabel('Modelo')
    axes[i].set_ylabel('Score')
    axes[i].set_ylim([0, 1])
    axes[i].grid(axis='y', alpha=0.3)
    axes[i].tick_params(axis='x', rotation=45)

# Elimina último subplot (vacío)
fig.delaxes(axes[-1])

plt.suptitle('Comparación Detallada de Modelos (Validación Cruzada)', 
            fontsize=16, weight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../../reports/figures/metrics_detail_cv.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Análisis de Sensibilidad: Enfoque en Fraude

Para detección AML, **Recall** y **F1-score** son críticos:
- **Recall alto**: Detectar máximos fraudes posibles (minimizar falsos negativos)
- **F1-score**: Balance entre precisión y recall

In [ ]:
# Análisis enfocado en fraude
print("\n" + "="*80)
print("ANÁLISIS ENFOCADO EN DETECCIÓN DE FRAUDE")
print("="*80)

fraud_metrics = cv_results[['recall', 'f1_score', 'auc_roc']].copy()
fraud_metrics['fraud_detection_score'] = (
    0.4 * fraud_metrics['recall'] +
    0.4 * fraud_metrics['f1_score'] +
    0.2 * fraud_metrics['auc_roc']
)

fraud_metrics = fraud_metrics.sort_values('fraud_detection_score', ascending=False)
print(fraud_metrics)
print("="*80)

best_fraud_model = fraud_metrics['fraud_detection_score'].idxmax()
print(f"\n🎯 MEJOR MODELO PARA FRAUDE: {best_fraud_model}")
print(f"   Fraud Detection Score: {fraud_metrics.loc[best_fraud_model, 'fraud_detection_score']:.4f}")
print(f"   (40% Recall + 40% F1 + 20% AUC-ROC)")

## 7. Conclusiones y Recomendaciones

### Hallazgos Clave:

1. **Validación Cruzada con SMOTE**:
   - ✅ Evita data leakage aplicando SMOTE solo en train
   - ✅ Resultados más robustos y generalizables
   - ✅ Reduce varianza en estimaciones

2. **Comparación de Modelos**:
   - XGBoost y Random Forest tienen desempeño comparable
   - Ambos superan significativamente a baselines simples
   - Ensemble methods capturan patrones no lineales en fraude

3. **Métricas para Fraude**:
   - **Recall** es crítico para no perder casos de lavado
   - **F1-score** balancea precisión y recall
   - **AUC-ROC** mide capacidad de discriminación general

### Recomendaciones:

1. **Producción**:
   - Usar modelo con mejor F1-score en fraude
   - Calibrar umbral según tolerancia a falsos positivos
   - Monitorear drift en features importantes

2. **Mejoras Futuras**:
   - ✅ Implementar modelo híbrido (OE3)
   - ✅ Agregar features derivadas (ratios, flags)
   - ✅ Explorar deep learning (LSTM para secuencias)
   - ✅ Tunear hiperparámetros con Optuna/GridSearch

3. **Regulatorio**:
   - Documentar umbral de decisión y justificación
   - Implementar explicabilidad (SHAP) para cada alerta
   - Auditoría periódica de falsos positivos/negativos